In [1]:
""" Testa allt mellan himmel och jord """
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import sys

pref = '../'

sys.path.append('C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')
import typ as tp
import travdata as td
import time
import concurrent.futures
from catboost import CatBoostClassifier, Pool
import pickle

from IPython.display import display

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 260)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 120)


In [2]:
import sklearn
metric_list = sorted(sklearn.metrics.SCORERS.keys())
(metric_list)


['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

## Läs in all data och tvätta    

In [5]:
###############################################################################
#                           Läs in all data och tvätta                        #
###############################################################################
def läs_in_data(förbered=True, lägg_till=True):
    v75 = td.v75(pref=pref)
    df_work = v75.get_df()
    # display(df_work.columns.to_list())
    
    df,enc = v75.förbered_data(missing_num=True)  # num hanteras av catboost
    df = v75.lägg_till_kolumner(df)
    return df,v75

In [ ]:
df,v75 = läs_in_data()
print(df.datum.max())

df[['datum','avd', 'bana','häst', 'rel_kr',	'streck_avst',	'rel_rank',	'h1_samma_bana','h1_bana',	'h2_samma_bana',	'h3_samma_bana',	'h1_samma_kusk',	'h2_samma_kusk',	'h3_samma_kusk']].tail(2)


Testa Next_datum

In [9]:
df, v75 = läs_in_data()
print('max_datum i df', df.datum.max())


../all_data.csv
Loading dataframe from the file: ../all_data.csv
plac finns i df
Handling missing categoricals
Handling missing numerics
Använd mod.skapa_modeller i stället
Använd mod.skapa_modeller i stället
Använd mod.skapa_modeller i stället
Använd mod.skapa_modeller i stället
Använd mod.skapa_modeller i stället
Använd mod.skapa_modeller i stället
max_datum i df 2023-01-28


In [40]:
gap=0  # om vi skall testa på en omgång som inte är nästa omgång
step=1 # hur många omgångar vi hoppa till för nästa test
alla_datum = df.datum.unique()
curr_datum = alla_datum[120]
# beräkna next_datum med step och gap
ix1 = np.where(alla_datum == curr_datum)[0][0]
next_datum = alla_datum[ix1 + step]  # nästa curr_datum learn until including
test_datum = alla_datum[ix1 + step+1 + gap] # nästa omgång att testa på

print('curr_datum', curr_datum, 'next_datum', next_datum, 'test_datum', test_datum)


curr_datum 2016-09-10 next_datum 2016-09-17 test_datum 2016-09-18


In [ ]:
def compute_weights(scores):
    """ Compute normalized weights to use to get the meta column. 
    Args:
        (list) scores: list of (i.e. F1 or AUC) scores from L2 models on validation set
    Returns:
        (list) weights: weights to use to get the meta column    
    """
    weights = scores / np.sum(scores)
    return weights


cat1L2_dict = {'AUC': 0.71232, 'F1': 0.34766, 'Acc': 0.80825, 'MAE': 0.19175}
cat1L2 = list(cat1L2_dict.values())
cat2L2 = [0.71488, 0.35033, 0.80898, 0.19102]
xgb1L2 = [0.59313, 0.22062, 0.77084, 0.22916 ]
xgb2L2 = [0.61568, 0.24473, 0.77818, 0.22182]

lists = [cat1L2, cat2L2, xgb1L2, xgb2L2]
AUC, F1, Acc, MAE = [], [], [], []
for l in lists:
    AUC.append(l[0])
    F1.append(l[1])
    Acc.append(l[2])
    MAE.append(l[3])

AUC=compute_weights(AUC)
F1=compute_weights(F1)
Acc=compute_weights(Acc)
MAE=compute_weights(MAE)
print('weights')
print(np.round(AUC, 4),np.round(np.sum(AUC),1))
print(np.round(F1, 4),np.round(np.sum(F1),1))
print(np.round(Acc, 4),np.round(np.sum(Acc),1))
print(np.round(MAE, 4),np.round(np.sum(MAE),1))
    



In [ ]:
# läs in xgb-modellerna och testa vilka features de är fit på
modeller = ['xgb1L1','xgb2L1', 'xgb1L2', 'xgb2L2']

for model in modeller:
    print(model, end=': ')
    with open(f'{pref}modeller/{model}.model', 'rb') as f:
        xgb_model = pickle.load(f)
    print(len(xgb_model.get_booster().feature_names))
    print(xgb_model.get_booster().feature_names)
    print()

# Testa random grid search xgb

In [ ]:
# random grid search för xgb
def randomGrid_xgb(use_model):
    
    ############ Inledande skit #####################################################
    modell_dict = {'cat1': {'#hästar': False, '#motst': 3, 'motst_diff': True, 'streck': False},
                   'cat2': {'#hästar': True,  '#motst': 3, 'motst_diff': True, 'streck': True},
                   'xgb1': {'#hästar': False, '#motst': 3, 'motst_diff': True, 'streck': False},
                   'xgb2': {'#hästar': True,  '#motst': 3, 'motst_diff': True, 'streck': True}
                   }


    L1_modeller = dict()
    L2_modeller = dict()

    for key, typ in modell_dict.items():
        L1_key = key + 'L1'
        model = tp.Typ(L1_key, typ['#hästar'], typ['#motst'],
                    typ['motst_diff'], typ['streck'])
        L1_modeller[L1_key] = model

        L2_key = key + 'L2'
        model = tp.Typ(L2_key, typ['#hästar'], typ['#motst'],
                    typ['motst_diff'], typ['streck'])
        L2_modeller[L2_key] = model

    print('keys and names i modeller')
    # print keys in dict modeller
    for key, typ in L1_modeller.items():
        assert key == typ.name, "key and value.name should be the same in modeller"
        print(key)

    print('keys and names i meta_modeller')
    for key, typ in L2_modeller.items():
        assert key == typ.name, "key and value.name should be the same in meta_modeller"
        print(key)

    # Läs in NUM_FEATURES.txt och CAT_FEATURES.txt
    with open(pref+'CAT_FEATURES.txt', 'r', encoding='utf-8') as f:
        cat_features = f.read().split()

    # läs in NUM_FEATURES.txt till num_features
    with open(pref+'NUM_FEATURES.txt', 'r', encoding='utf-8') as f:
        num_features = f.read().split()
    use_features = cat_features + num_features
    
    print('use_features:\n', use_features)
    
    ############ SLUT Inledande skit #################################################
    typ=L1_modeller[use_model]

    grid = {'max_depth': [3, 4, 5, 6, 7, 8],
            #  'iterations': [100, 200, 300, 400, 500],   n_estimators
             'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
              'n_estimators': [100, 200, 300, 400, 500],
              'reg_lambda': [0.1, 1, 2, 3, 4],
              'subsample': [0.5, 0.8, 1]}


    if not typ.streck:
        print('remove streck')
        use_features.remove('streck')

    
    df,_=v75.förbered_data(extra=True)
    X = typ.prepare_for_model(df.drop(['y'],axis=1))
    y = df.y.copy()
    
    # xgb_encoder till ENC
    with open(pref+'xgb_encoder.pkl', 'rb') as f:
        ENC = pickle.load(f)
        
    X, ENC = tp.prepare_for_xgboost(X, encoder=ENC)
    display(X[use_features].head(1))

    
    assert X[use_features].isnull().sum().sum() == 0, 'there are NaN values in cat_features'
    # assert that all cat_features are numeric
    non_numeric_columns = X[use_features].select_dtypes(exclude=[np.number]).columns
    assert len(non_numeric_columns)==0, f"Följande kolumner är inte numeriska: {non_numeric_columns}"

    tscv = TimeSeriesSplit(n_splits=5)

    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', random_state=2023)

    display(f'XGB Randomized search')
    grid_search_result = RandomizedSearchCV(model, param_distributions=grid,
                                                    cv=tscv.split(X[use_features]),
                                                    scoring=['roc_auc','neg_log_loss'],
                                                    random_state=2023,
                                                    refit='roc_auc',
                                                    n_jobs=-1,
                                                    verbose=1)
    grid_search_result.fit(X[use_features], y)

    return grid_search_result

In [ ]:
grid_search_result = randomGrid_xgb('xgb1L1')


In [ ]:
display(grid_search_result.best_params_)
display(grid_search_result.best_score_)
ix = grid_search_result.best_index_
display(-grid_search_result.cv_results_['mean_test_neg_log_loss'][ix])
display(grid_search_result.cv_results_['mean_test_roc_auc'] [ix])


In [ ]:
def fejka_resultat_tabell():
    ### init resutat-tabell
    df_resultat = pd.DataFrame(columns = ['datum', 't1_7', 't1_6', 't1_5', 't1_kostn', 't1_utd', 't1_vinst' ])
    df_resultat.set_index('datum',drop=True, inplace=True)
    df_resultat.loc['2019-08-01'] = [0, 0, 8,300,200, -100]
    df_resultat.loc['2019-08-08'] = [0, 0, 0,600,200, -400]
    df_resultat.loc['2019-08-15'] = [0, 1, 2,900,600, -300]
    df_resultat.loc['2019-08-22'] = [1, 2, 8,1200,1000, -200]
    df_resultat.sort_index(ascending=True,inplace=True)
    # 3. plotta
    print(df_resultat.head())
    # bigger plot
    df_resultat.plot(kind='line',  y='t1_vinst', rot=45, legend=True, figsize=(20,10))
    return df_resultat

df_resultat = fejka_resultat_tabell()

In [ ]:
## TESTA DETTA I IPYNB
# testar diverse lösningar till v75 (streamlit)
# import module for randomforrest
from sklearn.ensemble import RandomForestRegressor as rf

def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    # läs in streck_to_odds.pkl
    import pickle
    with open(pref+'rf_streck_odds.pkl', 'rb') as f:
        rf = pickle.load(f)
        
    if odds is None:
        o = rf.predict(streck.copy())
    else:
        o = rf.predict(streck.copy())

    # for each values > 40 in odds set to 1
    o[o > 40] = 1
    return (o*proba - (1-proba))/o

def compute_total_insats(veckans_rad):
    summa = veckans_rad.groupby('avd').avd.count().prod() / 2
    return summa

def beräkna_utdelning(datum, sjuor, sexor, femmor, df_utdelning):
    min_utdelning = df_utdelning.loc[df_utdelning.datum==datum,['7rätt', '6rätt','5rätt']]
    
    return (min_utdelning['7rätt'] * sjuor + min_utdelning['6rätt'] * sexor + min_utdelning['5rätt'] * femmor).values[0]

def ta_fram_rad(veckans_rad_, spik_strategi,kelly_strategi, max_cost=300, min_avst=0.25):
    """ Denna funktion tar fram en rad för typ-modeller (ej meta-modell)
    df nnehåller _en omgång_
    _spik_strategi_: None - inget, '1a' - forcera 1 spik, '2a' - forcera 2 spikar, '1b' - 1 spik endast om klar favorit, '2b' - spikar för endast klara favoriter 
    _kelly_strategi_: None - ingen kelly, 1 - kelly varannan gång om positiv
    """

    veckans_rad = veckans_rad_.copy()
    veckans_rad['välj'] = False   # inga rader valda ännu
    veckans_rad['spik'] = False   # inga spikar valda ännu
    # a) ta ut en häst i varje avd - markera valda i df
    
    for avd in veckans_rad.avd.unique():
        # max av proba i veckans_rad 
        max_proba = veckans_rad[veckans_rad.avd == avd]['proba'].max()
        veckans_rad.loc[(veckans_rad.avd == avd) & (veckans_rad.proba == max_proba), 'välj'] = True
    
    # b) leta 1-2 spikar om så begärs - markera valda i df
    spikad_avd = []
    if spik_strategi:
        print('spik_strategi', spik_strategi)
        assert spik_strategi in ['1a','1b','2a','2b'], "spik_strategi måste ha något av värdena i listn"
        # Hitta spik-kandidater
        if spik_strategi[0] in ['1','2']:
            spik1 = veckans_rad.nlargest(1,'proba').index[0]   # largest in dataset
            avd = veckans_rad.loc[spik1,'avd']
            no2 = veckans_rad.query("avd==@avd").nlargest(2, 'proba').index[1]  # second in avd
            print(f'hösta proba={veckans_rad.loc[spik1, "proba"]} i avd={avd}; no2 i avd={avd} är {veckans_rad.loc[no2,"proba"]}')
            avstånd = veckans_rad.loc[spik1, 'proba'] - veckans_rad.loc[no2, 'proba']
            print('avst', avstånd)
            if (spik_strategi[1] == 'b') and (avstånd > min_avst):
                print('strategi', spik_strategi[1], 'valde spik i avd',avd)
                # add avd to a list
                spikad_avd.append(avd)
                
                veckans_rad.loc[spik1,'spik'] = True
                veckans_rad.loc[spik1, 'välj'] = True
            elif spik_strategi[1] == 'a':
                print('strategi',spik_strategi[1], 'valde spik i avd',avd)
                spikad_avd.append(avd)
                veckans_rad.loc[spik1,['spik']]= True
                veckans_rad.loc[spik1, 'välj']= True
                
        if spik_strategi[0] == '2':
            spik2 = veckans_rad.nlargest(2,'proba').index[1] # second in dataset
            avd = veckans_rad.loc[spik2, 'avd']
            no2 = veckans_rad.query("avd==@avd").nlargest(2, 'proba').index[1]  # second in avd
            print(f'näst högsta proba={veckans_rad.loc[spik2, "proba"]} i avd={avd}; no2 i avd={avd} är {veckans_rad.loc[no2,"proba"]}')
            avstånd = veckans_rad.loc[spik2, 'proba'] - veckans_rad.loc[no2, 'proba']
            print('avst',avstånd)
            if (spik_strategi[1] =='b') and (avstånd > min_avst):
                print('strategi', spik_strategi[1], 'valde spik i avd', avd)    
                spikad_avd.append(avd)
                veckans_rad.loc[spik2, 'spik'] = True
                veckans_rad.loc[spik2, 'välj'] = True
            elif spik_strategi[1] == 'a':
                print('strategi', spik_strategi[1], 'i avd',avd)
                spikad_avd.append(avd)
                veckans_rad.loc[spik2,'spik'] = True
                veckans_rad.loc[spik2,'välj'] = True
                
    # c) sortera upp i proba-ordning. Om kelly skapa en sortering efter kelly-ordning
    veckans_rad = veckans_rad.sort_values(by=['proba'], ascending=False)
    veckans_rad = veckans_rad.reset_index(drop=True)
    if kelly_strategi == '1':
        veckans_kelly = veckans_rad.sort_values(by=['kelly'], ascending=False)
        veckans_kelly = veckans_kelly.reset_index(drop=True)
    
    cost = 0.5 # 1 rad
    while cost < max_cost:
        # d) plocka en och en - först proba sedan ev positiv kelly markera som valda i df
        curr_index = veckans_rad.query("välj==False and avd not in @spikad_avd").nlargest(1,'proba').index
        veckans_rad.loc[curr_index,'välj'] = True
        # e) avbryt vid 300:-
        cost = compute_total_insats(veckans_rad.query("välj==True"))
        if  cost > max_cost:
            veckans_rad.loc[curr_index, 'välj'] = False  # ta tillbaks den sist spelade
            break
        if kelly_strategi == '1' and veckans_rad.query("välj==False and avd not in @spikad_avd and kelly > 0").shape[0] > 0:
            curr_index = veckans_rad.query("välj==False and avd not in @spikad_avd and kelly > 0").nlargest(1,'kelly').index
            cost = compute_total_insats(veckans_rad.query("välj==True"))
            veckans_rad.loc[curr_index, 'välj'] = True
            if  cost > max_cost:
                veckans_rad.loc[curr_index, 'välj'] = False  # ta tillbaks den sist spelade
                break
    cost = compute_total_insats(veckans_rad.query("välj==True"))
    return veckans_rad, cost

def rätta_rad(df, datum, df_utdelning ):
    """
    Räkna ut antal 5:or, 6:or resp. 7:or
    Hämta ev utdelning
    Spara datum, resultat, utdelning och rad-kostnad
    """
    sjuor, sexor, femmor, utdelning = 0,0,0,0
    
    min_tabell = df[['y', 'avd', 'häst', 'rel_rank', 'välj']].copy()
    min_tabell.sort_values(by=['avd', 'y'], ascending=False,inplace=True)

    # 1. om jag har max 7 rätt
    if min_tabell.query('välj==True and y==1').y.sum() == 7:
        sjuor=1
        sexor = (min_tabell.groupby('avd').välj.sum()).sum()-7
        # antal femmor
        ant1 = min_tabell.query('avd==1 and välj==True').välj.sum()-1
        ant2 = min_tabell.query('avd==2 and välj==True').välj.sum()-1
        ant3 = min_tabell.query('avd==3 and välj==True').välj.sum()-1
        ant4 = min_tabell.query('avd==4 and välj==True').välj.sum()-1
        ant5 = min_tabell.query('avd==5 and välj==True').välj.sum()-1
        ant6 = min_tabell.query('avd==6 and välj==True').välj.sum()-1
        ant7 = min_tabell.query('avd==7 and välj==True').välj.sum()-1
        femmor = ant1*ant2+ant1*ant2+ant1*ant3+ant1*ant4+ant1*ant5+ant1*ant6+ant1*ant7 +\
                ant2*ant3+ant2*ant4+ant2*ant5+ant2*ant6+ant2*ant7 + \
                ant3*ant4+ant3*ant5+ant3*ant6+ant3*ant7 + \
                ant4*ant5+ant4*ant6+ant4*ant7 + \
                ant5*ant6+ant5*ant7 + \
                ant6*ant7

    # 2. jag har max 6 rätt
    if min_tabell.query('välj==True and y==1').y.sum() == 6:
        avd_fel = min_tabell.loc[((min_tabell.välj==False) & (min_tabell.y==1)),'avd'].values[0]
        print(min_tabell.query('avd== @avd_fel').välj.sum())
        sexor = min_tabell.query('avd==@avd_fel').välj.sum()
        # antal femmor
        femmor_fel, femmor_rätt = 0,0
        for avd in range(1,8):
            if avd == avd_fel:
                femmor_fel += min_tabell.loc[min_tabell.avd==avd_fel].välj.sum()
                
            femmor_rätt += min_tabell.query('avd==@avd and välj==True').välj.sum()-1
        print(f'femmor_rätt = {femmor_rätt} femmor_fel = {femmor_fel}')    
        femmor = femmor_fel * femmor_rätt

    # 3. jag har max 5 rätt
    if min_tabell.query('välj==True and y==1').y.sum() == 5:
        avd_fel = min_tabell.loc[((min_tabell.välj==False) & (min_tabell.y==1)),'avd'].values
        femmor = min_tabell.loc[min_tabell.avd==avd_fel[0]].välj.sum() * min_tabell.loc[min_tabell.avd==avd_fel[1]].välj.sum()
    
    # 4. utdelning 
    
    return sjuor, sexor, femmor, beräkna_utdelning(datum, sjuor,sexor,femmor, df_utdelning)

###############################################################################
#      main logic v75
def testa_main_v75():
    df_utdelning = pd.read_csv(pref+'/utdelning.csv')
    test1 = tp.Typ('test1',  True,    True,     False,       0,  False,          0,        False,    True, pref=pref)

    startdatum = '1900-01-01'
    # 0. ta fram startdatum  (datum=startdatum)
    curr_datix = len(df.datum.unique()) - 200      # ca 3 å3 tillbaks
    startdatum = df.datum.unique()[curr_datix]   # ca 3 år tillbaks
    datum = startdatum
    # 1. learn fram till datum
    print(f'learn fram till {datum}')
    X = df.query(f'datum < @datum').copy()
    y = X.y
    X = X.drop('y', axis=1)
    X_test = df.query(f'datum > @datum').copy()
    y_test = X_test.y
    X_test = X_test.drop('y', axis=1)
    X_curr = df.query(f'datum == @datum').copy()
    veckans_rad = X_curr[['datum','avd','häst','bana','kusk','streck','streck_avst','rel_rank','y']].copy()
    y_curr = X_curr.y
    X_curr = X_curr.drop(['y'], axis=1)

    params = {"depth": 2, "l2_leaf_reg": 3,"iterations": 500, "learning_rate": 0.008}
    model = test1.learn(X, y, X_test=X_test, y_test=y_test,save=True, params=params)
    print(X.shape)
    print(X_curr.shape)
    veckans_rad['proba'] = test1.predict(X_curr)
    veckans_rad['kelly'] = kelly(veckans_rad.proba, veckans_rad[['streck']], None)

    # 2. ta fram rad för datum, rätta och spara
    # inkluderar spik_strategi,kelly_strategi,
    veckans_rad, kostnad = ta_fram_rad(veckans_rad, '2b', '1', min_avst=0.3)

    sjuor, sexor, femmor, utdelning = rätta_rad(veckans_rad, datum, df_utdelning)
    print('kostnad',kostnad, 'utdelning', utdelning)

    display("SPARA RESLUTAT")
    # 3. plotta

        # loopa över olika setup

    # 4. startdatum+1
    # 5. gå till 1

    return datum, kostnad, utdelning, sjuor, sexor, femmor, veckans_rad

def testa_resultat(datum,kostnad, utdelning, sjuor, sexor, femmor):
    df_resultat = pd.DataFrame(
        columns=['datum', 't1_7', 't1_6', 't1_5', 't1_kostn', 't1_utd', 't1_vinst'])
    df_resultat.set_index('datum', drop=True, inplace=True)
    dict = {'t1_7': sjuor, 't1_6': sexor, 't1_5': femmor,
            't1_kostn': kostnad, 't1_utd': utdelning, 't1_vinst': utdelning-kostnad}
    dict = [sjuor, sexor, femmor, kostnad,  utdelning,  utdelning-kostnad]
    # set last row to dict
    df_resultat.loc[datum] = dict
    datum = '2022-07-08'
    df_resultat.loc[datum] = [sjuor+1, sexor, femmor,
                            kostnad,  utdelning,  utdelning-kostnad]

    df_resultat.sort_index(ascending=True, inplace=True)

    df_resultat

    return df_resultat


In [ ]:
datum ,kostnad, utdelning, sjuor, sexor, femmor, veckans_rad = testa_main_v75()
df_resultat = testa_resultat(datum,kostnad, utdelning, sjuor, sexor, femmor)
    

In [ ]:
def kolla_värdena(veckans_rad, sjuor, sexor, femmor, kostnad, utdelning):
    ##### kolla värdena
    print(kostnad)
    print('sjuor',sjuor,'sexor', sexor, 'femmor', femmor)
    print(len(veckans_rad.query("välj==True")))
    veckans_rad.query("välj==True").sort_values(by=['avd','proba'],ascending=[True,False])
    # veckans_rad.query("välj==True").sort_values(by='avd')
    # min_utdelning = df_utdelning.loc[df_utdelning.datum == datum, ['7rätt', '6rätt', '5rätt']]
    # print(min_utdelning)
    # min_utdelning['7rätt'] * sjuor + min_utdelning['6rätt'] * sexor + min_utdelning['5rätt'] * femmor

kolla_värdena(veckans_rad, sjuor, sexor, femmor, kostnad, utdelning)

In [ ]:
def diff_meta_proba(X_):
    sm = X_.copy()
    # select the highest meta_predict per avd
    sm['first'] = sm.groupby('avd')['meta_predict'].transform(
        lambda x: x.nlargest(2).reset_index(drop=True)[0])
    sm['second'] = sm.groupby('avd')['meta_predict'].transform(
        lambda x: x.nlargest(2).reset_index(drop=True)[1])

    sm = sm.query("(first==meta_predict or second==meta_predict)").copy()
    sm['diff'] = sm['first'] - sm['second']

    # drop duplicates per avd
    sm = sm.drop_duplicates(subset='avd', keep='first')

    sm.sort_values(by='diff', ascending=False, inplace=True)
    # sm.to_csv('mesta_diff_per_avd.csv')
    return sm


def diff_streck(df, curr_datum='2022-07-30'):
    larg = df.query('datum == curr_datum').groupby(
        ['avd'])['streck'].nlargest(2)
    larg = larg.reset_index().drop('level_1', axis=1)

    # select the first row for each avd
    first = larg.groupby(['avd']).head(1).set_index('avd')
    second = larg.groupby(['avd']).tail(1).set_index('avd')

    diff = first.sub(second, axis=0)
    # print(diff)
    return diff


In [ ]:
def hx_samma_bana(df_, datum='2022-07-30'):
    df = df_.copy()
    return df[df.datum == datum].bana == df[df.datum == datum].h1_bana
# hx_samma_bana(df)


def hx_samma_kusk(df_, datum='2022-07-30'):
    df = df_.copy()
    return df[df.datum == datum].kusk == df[df.datum == datum].h1_kusk


hx_samma_kusk(df).head()


In [ ]:
# ta ut max srteck per avd
def max_streck_per_avd(df_):
    df = df_.copy()
    df['max_streck'] = df.groupby(
        ['datum', 'avd']).streck.transform(lambda x: x.max())

    df['streck_avst'] = df.max_streck - df.streck
    df.drop(['max_streck'], axis=1, inplace=True)
    return df


max_streck_per_avd(df)[['häst', 'avd', 'streck_avst', 'streck', ]]


In [ ]:
# plats i streck per avd
def plats_i_streck_per_avd(df_):
    df = df_.copy()
    # sortera streck per datum,avd
    # df = df.sort_values(by=['datum', 'avd', 'streck'])
    # ranking per avd
    df['rank'] = df.groupby(['datum', 'avd'])['streck'].rank(ascending=False, method='dense')
    df.sort_values(by=['datum', 'avd', 'rank'], inplace=True)
    return df
        
        


plats_i_streck_per_avd(df)[['avd','häst','streck','streck_avst','rank']]


In [ ]:
# what row has max kr in total in df
df['välj']=False
kelly_strategi = '1'
df.loc[df.nlargest(2,'rel_kr').index,'välj'] = True

df.loc[df.query("välj==False").nlargest(1,'rel_kr').index,'välj'] = True

if df.query("välj==False and rel_rank > 0").shape[0] > 0:
    df.loc[df.query("välj==False and rel_rank > 0").nlargest(1, 'rel_rank').index, 'välj'] = True

df.välj.value_counts()



In [ ]:
# # create a new row by appending the new values to df_resultat
# last_row = df_resultat.iloc[-1] + [sjuor,sexor,femmor,kostnad,utdelning,utdelning-kostnad]
# datum='2022-08-26'
# # add the new row to df_resultat with loc datum
# df_resultat.loc[datum] = last_row

# utred varför rf blir så dålig

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

# write the scores
def display_scores(y_true, y_pred, spelade):
    print('AUC', round(roc_auc_score(y_true, y_pred), 5), 'F1', round(f1_score(y_true, y_pred), 5), 'Acc', round(
        accuracy_score(y_true, y_pred), 5), 'MAE', round(mean_absolute_error(y_true, y_pred), 5), '\n', spelade)
    return roc_auc_score(y_true, y_pred)


def find_threshold(y_pred, fr, to, margin):
    """ hitta threshold som ger 2.5 spelade per avdelning """
    thresh = 0
    cnt = 0
    # make a binary search
    while cnt < 1000:
        thresh = (fr + to) / 2
        antal_spelade_per_avd = 12 * sum(y_pred > thresh)/len(y_pred)
        if (antal_spelade_per_avd > (2.5 - margin)) and (antal_spelade_per_avd < (2.5 + margin)):
            break

        if antal_spelade_per_avd > 2.5:
            fr = thresh-0.00001
        else:
            to = thresh+0.00001
        cnt += 1

    print('ant', cnt, 'thresh', round(thresh, 4))
    if cnt >= 1000:
        print('threshold not found', 'fr', round(fr, 6), 'to', round(to, 6))

    return thresh

def plot_confusion_matrix(y_true, y_pred, typ, fr=0.0, to=0.9, margin=0.001):
    thresh = round(find_threshold(y_pred,fr,to,margin), 4)
    print(f'Threshold: {thresh}\n')
    y_pred = (y_pred > thresh).astype(int)
    # confusion_matrix_graph(y_true, y_pred, f'{typ} threshold={thresh}')

    #### Sedan: confusion matrix graph ####
    title = f'{typ} threshold={thresh}'
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred,)
    fig, ax = plt.subplots()
    # make it bigger
    fig.set_size_inches(10, 10)
    sns.set(font_scale=2.0)
    sns.heatmap(cm/np.sum(cm), annot=True, fmt=".2%", linewidths=.5,
                square=True, cmap='Blues_r')

    # increase font size
    plt.rcParams['font.size'] = 20
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title(title)
    # plot fig
    plt.show()
    
    _=display_scores(y_true, y_pred, f'spelade per lopp: {round(12 * sum(y_pred)/len(y_pred),4)}')

    

In [ ]:
rf_train = pd.read_csv(pref + 'rf_train.csv')
rf_validate = pd.read_csv(pref + 'rf_validate.csv')
rf_y_pred = pd.read_csv(pref + 'rf_y_pred.csv')
rf_y_true = pd.read_csv(pref + 'rf_y_true.csv')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import json
with open(pref+'optimera/params_rf.json', 'r') as f:
        params = json.load(f)
        params = params['params']
        # st.write(params)

rf_model = RandomForestClassifier(**params, n_jobs=6, random_state=2022)
rf_fit=rf_model.fit(rf_train.drop('y',axis=1), rf_train.y)


In [ ]:
new_pred = rf_fit.predict_proba(rf_validate)[:,1]
print('förbereder rf plot')
plot_confusion_matrix(rf_y_true, new_pred,
                      'rf', fr=0.0, to=1.0, margin=0.001)
